# Import libraries

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as mtp
import pandas as pd
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
# Loading dataset into data_set variable
data_set = pd.read_csv('./vehicle_data.csv')

In [ ]:
# Previewing data set column and row count
data_set.shape